In [1]:
# Packages
import pandas as pd
from helpers.preprocessing import *
import ast
import numpy as np

In [2]:
ROOT_DIR  = ''
IMAGE_DIR = './data/images' # directory to save images
LABEL_DIR = './data/labels' # directory to save labels
RAW_IMAGE_DIR = './data/train_images'
N_samples = 220
IMAGE_HEIGHT = 1280
IMAGE_WIDTH = 720


In [3]:
# Read labels and add column n annotations
labels_src = 'data/train.csv'

labels = pd.read_csv(labels_src)
labels['n_annotations'] = count_n_annotations(labels)
labels = labels.apply(get_path,  args=(ROOT_DIR, IMAGE_DIR, LABEL_DIR), axis=1)
labels['annotations'] = labels['annotations'].apply(lambda x: ast.literal_eval(x))
labels['bboxes'] = labels.annotations.apply(get_bbox)

In [4]:
# Random sample images for training and testing from video_0 folder
if len(os.listdir(IMAGE_DIR)) != N_samples:
    print('REMOVING OLD FILES, COPYING NEW ONES')
    for file in os.listdir(IMAGE_DIR):
        os.remove(os.path.join(IMAGE_DIR, file))
    subsample_images(os.path.join(RAW_IMAGE_DIR, 'video_0'),'data/images', labels, N_samples, 0.5)

In [5]:
# Subset df of labels to subsample of images
df = labels[labels.image_path.isin([IMAGE_DIR+'/'+ image_path for image_path in os.listdir(IMAGE_DIR)])]

In [6]:
# Df to Yolo Format
df_yolo = yolo_formatting(df)

# Add class label
df_yolo.head()

c:\Users\Lukas Mahieu\Desktop\Personal Projects\ProtectBarrierReef\helpers\preprocessing.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yolo_df['bbox'] = bboxes


,label_path,x,y,width,height
0,./data/labels/video_0_15.txt,0.00000,0.00000,0.00000,0.00000
1,./data/labels/video_0_18.txt,0.45469,0.31806,0.03906,0.04444
2,./data/labels/video_0_23.txt,0.44062,0.32083,0.04531,0.06667
3,./data/labels/video_0_43.txt,0.44453,0.23958,0.07187,0.10694
4,./data/labels/video_0_43.txt,0.50234,0.35139,0.05625,0.05833


In [8]:
# Output yolo format to text file per image
for i, row in df_yolo.iterrows():
    with open(row.label_path, 'a') as f:
        f.write('0 ' + str(row.drop('label_path').values)[1:-1] +'\n') # 0 = single class label